# Wildfire Prediction (TensorFlow)

This Jupyter Notebook serves as a reimplementation of the best-performing MLP and CNN from `wildfire_prediction.ipynb`. They will be reimplemented in TensorFlow so that they may be deployed as part of a wildfire detection app. 

Link to dataset: https://www.kaggle.com/datasets/brsdincer/wildfire-detection-image-data

Image Specifications:
- Type: `.jpg`
- Dimensions: 250 x 250 pixels
- Horizontal/vertical resolution: 96 dpi
- Bit depth: 24

In [81]:
import tensorflow as tf
from tensorflow.data import AUTOTUNE
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [82]:
print(tf.__version__)

2.19.0


In [83]:
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - 0.5) / 0.5
    return image, label

In [84]:
# Create train, validation, and test sets
tf.random.set_seed(42)

# Using 0.2005 for validation split ratio so that we stay consistent with using 1465 images for training and 367 images for validation
train_set = tf.keras.utils.image_dataset_from_directory(
    'forest_fire/Training and Validation',
    seed=123,
    validation_split=0.2005,
    subset='training',
    image_size=(250, 250),
    batch_size=64
).map(normalize).shuffle(buffer_size=1000, seed=42).prefetch(AUTOTUNE)

val_set = tf.keras.utils.image_dataset_from_directory(
    'forest_fire/Training and Validation',
    seed=123,
    validation_split=0.2005,
    subset='validation',
    image_size=(250, 250),
    batch_size=64
).map(normalize).prefetch(AUTOTUNE)

test_set = tf.keras.utils.image_dataset_from_directory(
    'forest_fire/Testing',
    labels='inferred',
    label_mode='int',
    image_size=(250, 250),
    batch_size=64,
    shuffle=False
).map(normalize).prefetch(AUTOTUNE)

Found 1832 files belonging to 2 classes.
Using 1465 files for training.
Found 1832 files belonging to 2 classes.
Using 367 files for validation.
Found 68 files belonging to 2 classes.


In [85]:
# Best performing MLP model from model testing (mlp_2)
mlp_2 = Sequential([
    Flatten(input_shape=(250, 250, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(2)
])

In [86]:
mlp_2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [87]:
mlp_2.fit(
    train_set,
    validation_data=val_set,
    epochs=20
)

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 131ms/step - accuracy: 0.8150 - loss: 8.9061 - val_accuracy: 0.8638 - val_loss: 4.4011
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.9263 - loss: 1.6419 - val_accuracy: 0.9455 - val_loss: 1.6376
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.9392 - loss: 0.8838 - val_accuracy: 0.9264 - val_loss: 1.9877
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.9352 - loss: 0.9496 - val_accuracy: 0.9019 - val_loss: 2.3305
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.9625 - loss: 0.5387 - val_accuracy: 0.9292 - val_loss: 1.9884
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.9693 - loss: 0.2391 - val_accuracy: 0.9537 - val_loss: 1.3256
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.9652 - loss: 0.3402 - val_accuracy: 0.9455 - val_loss: 1.5165
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.9700 - loss: 0.3525 - val_accuracy: 0.9428

In [88]:
mlp_2.evaluate(test_set)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8676 - loss: 5.8111


[5.811068534851074, 0.8676470518112183]

In [27]:
# Best performing CNN model from model testing (cnn_9)
cnn_9 = Sequential([
    Conv2D(32, kernel_size=7, padding='same', activation='relu', input_shape=(250, 250, 3)),
    MaxPooling2D(pool_size=2),
    Conv2D(64, kernel_size=7, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Conv2D(128, kernel_size=7, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(2)
])

/Users/jonathantruong/miniforge3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [72]:
cnn_9.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
cnn_9.fit(
    train_set,
    validation_data=val_set,
    epochs=20
)

In [ ]:
cnn_9.evaluate(test_set)